In [1]:
filelines = []

with open('./input/Lectionary 2021.txt', 'r') as txtfile:
    filelines = txtfile.readlines()
    
len(filelines)

10373

In [2]:
from enum import Enum
import re

class Section(Enum):
    UNKNOWN = 0,
    MONTH = 1
    DAY = 2,
    DAILY_EUCHARISTIC_L = 3,
    FOR_ALL_SAINTS = 4,
    MORNING_PRAYER = 5,
    EVENING_PRAYER = 6,
    REVISED_COMMON_L = 7,
    OTHER_READINGS_MORNING = 8,
    OTHER_READINGS_EVENING = 9


def isDaySection(line, currentSection):
    match = re.search("^(SUN|MON|TUE|WED|THU|FRI|SAT)\\b", line)
    if match:
        return True

    match = re.search("^[0-9]+(W|G|V|v|w|r|R| )*$", line)
    isDayNumber = match and currentSection == Section.DAY
    return True if isDayNumber else False

def isMonthSection(line):
    match = re.search("^(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER) ", line)
    return True if match else False

def isOtherReadingsMorning(line, previousLine):
    if(re.search("^Other Readings", previousLine) and re.search("^Morning", line)):
        return True 
    return False

def isOtherReadingsEvening(line, previousLine):
    if(re.search("^Other Readings", previousLine) and re.search("^Evening", line)):
        return True 
    return False
    
def getSection(line, previousLine = None, currentSection = Section.UNKNOWN):
    if(isDaySection(line, currentSection)):
        return Section.DAY
    if(isMonthSection(line)):
        return Section.MONTH
    if(isOtherReadingsMorning(line, previousLine)):
        return Section.OTHER_READINGS_MORNING
    if(isOtherReadingsEvening(line, previousLine)):
        return Section.OTHER_READINGS_EVENING
    
    return Section.UNKNOWN
    

In [3]:
days = {}

print(isMonthSection("NOVEMBER 2020"))
print(isDaySection("2V", Section.DAY))
print(isOtherReadingsEvening("Evening", "Other Readings"))

print(getSection("NOVEMBER 2020", None))
    
    

True
True
True
Section.MONTH


In [4]:
import json

days = []

def goThroughFile():
    previousLine = None
    currentSection = Section.MONTH
    widerSection = None
    
    currentMonth = "NOVEMBER 2020"
    currentDayOfWeek = "SAT"
    currentDay = "28"
    saints = None
    
    otherReadingsMorning = []
    otherReadingsEvening = []
    
    for line in filelines:
        
        line = line.replace('\n', '')     
        newSection = getSection(line, previousLine, currentSection)
        #print(f"{newSection} {line} ({widerSection})")
           
        hitStartOfNextDay = newSection == Section.MONTH or (newSection == Section.DAY and currentSection != Section.MONTH and currentSection != Section.DAY)
        if(hitStartOfNextDay and currentDayOfWeek):
            #summarise
            dayData = {
                'date': f"{currentDayOfWeek} {currentDay} {currentMonth}",
                'dayDescription': saints,
                'otherReadingsMorning':otherReadingsMorning,
                'otherReadingsEvening':otherReadingsEvening
            }       
            days.append(dayData)
            
            #clear the data
            currentDayOfWeek = None
            currentDay = None
            otherReadingsMorning = []
            otherReadingsEvening = []      
        
        if(newSection == Section.MONTH):  
            currentMonth = line
            widerSection = None
        elif (newSection == Section.DAY and currentSection != Section.DAY):
            currentDayOfWeek = line[0:3]
            saints = line[3:]
            widerSection = Section.DAY
        elif (newSection == Section.DAY and currentSection == Section.DAY):
            match = re.search("^[0-9]+", line)
            if(match):
                currentDay = match.group(0) #the whole match
            widerSection = None
        elif (newSection == Section.OTHER_READINGS_MORNING):
            widerSection = Section.OTHER_READINGS_MORNING
        elif (newSection == Section.OTHER_READINGS_EVENING):
            widerSection = Section.OTHER_READINGS_EVENING
        elif (newSection == Section.UNKNOWN and widerSection == Section.OTHER_READINGS_MORNING and line != "Other Readings"):
            otherReadingsMorning.append(line)
        elif (newSection == Section.UNKNOWN and widerSection == Section.OTHER_READINGS_EVENING):
            otherReadingsEvening.append(line)
          
        currentSection = widerSection if widerSection else newSection
        previousLine = line
              

In [5]:
days = []
goThroughFile()


def checkDate(currentDayDate, previousDayDate):
    daysOfWeek = ["MON", "TUE", "WED", "THU", "FRI", "SAT", "SUN"]

    # check for None
    if("None" in currentDayDate):
        return False
    
    dayComponents = currentDayDate.split(" ")    
    if(len(dayComponents) != 4):
        return False
    
    if not previousDayDate:
        return True
    
    previousDayComponents = previousDayDate.split(" ")
    
    # check day
    try:
        difference = int(dayComponents[1]) - int(previousDayComponents[1])
        if difference == 1 or ( dayComponents[1] == "1" and dayComponents[2] != previousDayComponents[2]):
            return True
        else:
            # if today happens more than once and yesterday does too, then we know we have an alternative set of days
            countOfToday = sum([day['date'] == currentDayDate for day in days])
            countOfPreviousDay = sum([day['date'] == previousDayDate for day in days])
            
            return countOfToday == countOfPreviousDay and countOfToday > 1
            
    except ValueError:
        return False
    
    # check day of week
    dayOfWeek = dayComponents[0]
    previousDayOfWeek = previousDayComponents[0]
    difference = daysOfWeek.index(dayOfWeek) - daysOfWeek.index(previousDayOfWeek)
    if(difference not in [1, -6]):
        return False
        
    return True
    

for index, value in enumerate(days):
    dayDate = days[index]['date']
    previousDayDate = days[index - 1]['date'] if index > 0 else None
    
    warningPrefix = '\033[91m'
    okPrefix = '\033[92m'
    
    isOk = checkDate(dayDate, previousDayDate)
    if isOk:
        print(f"{okPrefix}{dayDate}")
    else:
        print(f"{warningPrefix}{dayDate}")
    
    

SAT 28 NOVEMBER 2020
SUN 29 NOVEMBER 2020
MON 30 NOVEMBER 2020
TUE 1 DECEMBER 2020
WED 2 DECEMBER 2020
THU 3 DECEMBER 2020
FRI 4 DECEMBER 2020
SAT 5 DECEMBER 2020
SUN 6 DECEMBER 2020
MON 7 DECEMBER 2020
TUE 8 DECEMBER 2020
WED 9 DECEMBER 2020
THU 10 DECEMBER 2020
FRI 11 DECEMBER 2020
SAT 12 DECEMBER 2020
SUN 13 DECEMBER 2020
MON 14 DECEMBER 2020
TUE 15 DECEMBER 2020
WED 16 DECEMBER 2020
THU 17 DECEMBER 2020
FRI 18 DECEMBER 2020
SAT 19 DECEMBER 2020
SUN 20 DECEMBER 2020
MON 21 DECEMBER 2020
TUE 22 DECEMBER 2020
WED 23 DECEMBER 2020
THU 24 DECEMBER 2020
FRI 25 DECEMBER 2020
SAT 26 DECEMBER 2020
SUN 27 DECEMBER 2020
MON 28 DECEMBER 2020
TUE 29 DECEMBER 2020
WED 30 DECEMBER 2020
THU 31 DECEMBER 2020
FRI 1 JANUARY 2021
FRI 1 JANUARY 2021
SAT 2 JANUARY 2021
SUN 3 JANUARY 2021
MON 4 JANUARY 2021
TUE 5 JANUARY 2021
WED 6 JANUARY 2021
THU 7 JANUARY 2021
FRI 8 JANUARY 2021
SAT 9 JANUARY 2021
SUN 3 JANUARY 2021
MON 4 JANUARY 2021
TUE 5 JANUARY 2021
WED 6 JANUARY 2021
THU 7 JANUARY 2021
FRI 8 JANU

# Stop
Only continue once the above statement prints all the dates correctly.  If it looks like a rare typo (like Sun instead of SUN), just tweak the document.  If it looks like a common thing, try change the script.

In [6]:
foundDay = [day for day in days if day['date'] == "SUN 10 JANUARY 2021"]
print(foundDay)

[{'date': 'SUN 10 JANUARY 2021', 'dayDescription': ' BAPTISM OF THE LORD', 'otherReadingsMorning': ['Ps 89:19-29', '1 Samuel 16.1-', '3,13', 'John 1:29-34'], 'otherReadingsEvening': ['Ps 46;(47)', 'Isa 42:1-9', 'Eph 2:1-10', 'Celebrating Common Prayer: Form 5 during Epiphanytide', 'Collect for the Baptism of the Lord', 'Beloved Son of God,', 'whose Way was revealed in humility, through water and Spirit;', 'you received a baptism of repentance.', 'Empower all who are baptised in the name of the Father, Son and Holy Spirit', 'to embrace that commissioning', 'and to serve you faithfully in love.', 'For you are alive and reign with the Father,', 'in the unity of the Holy Spirit,', 'one God, now and forever. Amen.']}]


In [16]:
def mapToDictionary(psalm, oldTestament, newTestament, apocryphal = None):
    return {
        "psalm": psalm,
        "oldTestament": oldTestament,
        "newTestament": newTestament,
        "apocryphal": apocryphal
    }

#def removeExtraInformationAndMapToDictionary(list):
    
def removeOrAndTrim(reading):
    return reading.replace(" or")

def resolveAlternativesAndMapToDictionary(list):   
    apocryphal = ["Tobit", "1 Macc", "2 Macc", "Sirach", "Baruch", "Wisd"]
    for book in apocryphal:
        if book in list[1]:
            return mapToDictionary(list[0], list[2], list[3], list[1])
        elif book in list[2]:
            return mapToDictionary(list[0], list[1], list[3], list[2])

    return []
            
def mapThreeReadingsToDictionary(list):
    return mapToDictionary(list[0], list[1], list[2])
    

def processOtherReadings(isMorning):
    
    suffix = "Morning" if isMorning else "Evening"
    otherReadingsKey = f"otherReadings{suffix}"
    daysWithOtherReadings = [day for day in days if len(day[otherReadingsKey])]
    
    for day in daysWithOtherReadings:
        readingsList = day[otherReadingsKey]
        
        otherReadingsDictionary = None
        if(len(readingsList) == 3):
            otherReadingsDictionary = mapThreeReadingsToDictionary(readingsList)
        elif (len(readingsList) == 4):
            otherReadingsDictionary = resolveAlternativesAndMapToDictionary(readingsList)
        
        print(otherReadingsDictionary)
           
processOtherReadings(True)

{'psalm': 'Ps 44', 'oldTestament': 'Isa 2:1-5', 'newTestament': 'Luke 12:35-48', 'apocryphal': None}
{'psalm': 'Ps 47;147:1-12', 'oldTestament': 'Ezek 47:1-12 or', 'newTestament': 'John 12:20-32', 'apocryphal': 'Sirach 14:20-27'}
{'psalm': 'Ps 80', 'oldTestament': 'Zeph 3:14-20', 'newTestament': 'Luke 1:5-20', 'apocryphal': 'Baruch 5:1-9 or'}
{'psalm': 'Ps 50:1-6,62', 'oldTestament': 'Isa 12', 'newTestament': 'Luke 1:57-66', 'apocryphal': None}
{'psalm': 'Ps 144', 'oldTestament': 'Isa 7:10-16', 'newTestament': 'Rom 1:1-7', 'apocryphal': None}
{'psalm': 'Ps 110, (117)', 'oldTestament': 'Isa 62:1-5', 'newTestament': 'Matt 1:18-25', 'apocryphal': None}
{'psalm': 'Ps 13;(31:1-9;150)', 'oldTestament': 'Jer 26:12-15', 'newTestament': 'Acts 6', 'apocryphal': None}
None
{'psalm': 'Ps 36,(146)', 'oldTestament': 'or Gen 37:13-20', 'newTestament': 'Matt 18:1-10', 'apocryphal': 'Baruch 4:21-27'}
{'psalm': 'Ps 103,(150)', 'oldTestament': 'Gen 17:1-13', 'newTestament': 'Rom 2:17-29', 'apocryphal': N

In [ ]:
sundays = [day for day in days if "SUN " in day['date']]

correctMorningSundays = [day for day in sundays if len(day['otherReadingsMorning']) == 3]


incorrectMorningSundays = [day for day in sundays if len(day['otherReadingsMorning']) != 3]

for day in correctMorningSundays:
    readings = "\t\t".join(day['otherReadingsMorning'])
    print(f"{day['date']} - \t{readings}")

print(len(incorrectMorningSundays))
for day in incorrectMorningSundays:
    readings = "; ".join(day['otherReadingsMorning'])
    print(f"{day['date']} - {readings}")